In [1]:
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif,  chi2
from sklearn import svm
from scipy.sparse import csr_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import csv


In [2]:
dom1 = []
dom2 = []
test_set = []
with open('data/domain1_train.json', 'r') as file:
    for line in file:
        dom1.append(json.loads(line))
        
with open('data/domain2_train.json', 'r') as file:
    for line in file:
        dom2.append(json.loads(line))
        
sam = pd.read_csv("data/sample.csv")

with open('data/test_set.json', 'r') as file:
    for line in file:
        test_set.append(json.loads(line))
        
dom1 = pd.DataFrame.from_dict(dom1)
dom2 = pd.DataFrame.from_dict(dom2)
dom2 = dom2[dom2['text'].apply(len) > 0]
test_set = pd.DataFrame.from_dict(test_set)
        

In [3]:
print(len(dom1))
print(len(dom2))
print(len(sam))
print(len(test_set))

19500
14899
1000
1000


In [4]:
dom1['label'].value_counts()
print(len(dom2.loc[dom2['label']==0]))
print(len(dom2.loc[dom2['label']==1]))

12750
2149


## Resampling

In [5]:
#让dom2的label，model分布都平均，差不多307一组，st 307行是label 1 标为model 5，再从dom1里取307个label1和307个label0，人工的model是nan

dom2_0 = dom2.loc[dom2['label']==0]
dom2_1 = dom2.loc[dom2['label']==1]

length_dom2_1 = len(dom2_1)
mol_num = len(dom2['model'].value_counts())
sam_dom2_0 = dom2_0.groupby('model').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sam_dom2_0.reset_index(drop=True, inplace=True)
dom2_new = result = pd.concat([dom2_1, sam_dom2_0])

sampled_dom1 = dom1.groupby('label').apply(lambda x: x.sample(n=int(length_dom2_1/mol_num)))
sampled_dom1.reset_index(drop=True, inplace=True)

df = result = pd.concat([sampled_dom1, dom2_new])

In [6]:
print(df['label'].value_counts())
print(df['model'].value_counts())


label
0    2456
1    2456
Name: count, dtype: int64
model
0.0    307
1.0    307
2.0    307
3.0    307
4.0    307
5.0    307
6.0    307
Name: count, dtype: int64


In [7]:
df = df.sample(frac=1).reset_index(drop=True)

In [8]:
df = df.drop(['model'], axis=1)

In [12]:

def to_sparse_matrix(sequences, vocab_size):
    indptr = [0]
    indices = []
    data = []
    for seq in sequences:
        feature_counter = {}
        for index in seq:
            if index != 0:  # Skip 0s, other words
                if index not in feature_counter:
                    feature_counter[index] = 1
                else:
                    feature_counter[index] += 1
        indices.extend(feature_counter.keys())
        data.extend(feature_counter.values())
        indptr.append(len(indices))
    n_samples = len(sequences)  # 获取样本数
    return csr_matrix((data, indices, indptr), dtype=int, shape=(n_samples, vocab_size + 1))

vocab_size = 4999
X = to_sparse_matrix(df['text'], vocab_size)


In [29]:
# X = df['text'].to_list()
y = df['label'].to_list()

# selector = SelectKBest(chi2, k=1000)
# X_new = selector.fit_transform(X, y)

In [42]:

# from torch.nn.utils.rnn import pad_sequence

# # 假设X_series是包含数字化文本序列的Pandas Series
# X_series = df['text']

# # 找到最长的序列长度
# max_seq_length = max(len(seq) for seq in X_series)

# # 填充序列，将其转换为PyTorch张量
# X_padded = pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in X_series], 
#                         batch_first=True, padding_value=0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 将数据转换为PyTorch张量
X_train_tensor = torch.tensor(X_train.toarray(), dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.toarray(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)



In [43]:
X_train_tensor


tensor([[ 0,  1,  5,  ...,  0,  0,  0],
        [ 0,  5,  3,  ...,  0,  0,  0],
        [ 0,  5,  2,  ...,  0,  0,  0],
        ...,
        [ 0, 15,  4,  ...,  0,  0,  0],
        [ 0, 14, 10,  ...,  0,  0,  0],
        [ 0, 34, 51,  ...,  0,  0,  0]])

In [44]:
# 创建数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

batch_size = 64  # 适当选择批量大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [12]:
# from sklearn.base import BaseEstimator
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset

# class TorchEstimator(BaseEstimator):
#     def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, output_dim,
#                  learning_rate=0.001, batch_size=64, num_epochs=10):
#         self.vocab_size = vocab_size
#         self.embedding_dim = embedding_dim
#         self.hidden_dim = hidden_dim
#         self.num_layers = num_layers
#         self.output_dim = output_dim
#         self.learning_rate = learning_rate
#         self.batch_size = batch_size
#         self.num_epochs = num_epochs

#     def fit(self, X, y):
#         # 将数据转换为PyTorch张量并创建数据加载器
#         X_tensor = torch.tensor(X, dtype=torch.long)
#         y_tensor = torch.tensor(y, dtype=torch.float32)
#         dataset = TensorDataset(X_tensor, y_tensor)
#         loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

#         # 创建模型
#         model = self.create_model()
#         criterion = nn.BCELoss()
#         optimizer = optim.Adam(model.parameters(), lr=self.learning_rate)

#         # 训练模型
#         for epoch in range(self.num_epochs):
#             model.train()
#             for batch in loader:
#                 text, labels = batch
#                 optimizer.zero_grad()
#                 predictions = model(text)
#                 loss = criterion(predictions.squeeze(1), labels)
#                 loss.backward()
#                 optimizer.step()

#     def predict(self, X):
#         # 将数据转换为PyTorch张量
#         X_tensor = torch.tensor(X, dtype=torch.long)

#         # 创建模型并加载之前训练的最佳模型权重
#         model = self.create_model()
#         model.load_state_dict(torch.load('best_model_weights.pth'))

#         # 预测
#         model.eval()
#         with torch.no_grad():
#             predictions = model(X_tensor)
#             return (predictions.numpy() > 0.5).astype(int)

#     def create_model(self):
#         # 创建并返回PyTorch模型
#         model = LSTMModel(self.vocab_size, self.embedding_dim, self.hidden_dim, self.num_layers, self.output_dim)
#         return model


In [13]:
# from sklearn.model_selection import GridSearchCV

# # 定义超参数空间
# param_grid = {
#     'learning_rate': [0.001, 0.01, 0.1],
#     'embedding_dim': [64, 128, 256],
#     'hidden_dim': [128, 256, 512],
#     'num_layers': [1, 2, 3],
#     'batch_size': [32, 64, 128]
# }

# # 创建 GridSearchCV 对象并进行超参数搜索
# estimator = TorchEstimator(vocab_size, embedding_dim, hidden_dim, num_layers, output_dim)
# grid_search = GridSearchCV(estimator, param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # 获取最佳模型并进行预测
# best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)


In [33]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded)
        hidden = hidden[-1, :, :]  # 取最后一层的隐状态
        out = self.fc(hidden)
        return self.sigmoid(out)


In [45]:
vocab_size = 5000  # 词汇表大小，根据你的数据集设置
embedding_dim = 128  # 嵌入维度，可以根据需要调整
hidden_dim = 256  # LSTM隐藏层维度，可以根据需要调整
num_layers = 1  # LSTM层数，可以根据需要调整
output_dim = 1  # 输出维度，因为是二进制分类，所以是1

model = RNNModel(vocab_size, embedding_dim, hidden_dim, num_layers, output_dim)
criterion = nn.BCELoss()  # 二进制交叉熵损失
optimizer = optim.Adam(model.parameters(), lr=0.01)  # 选择优化器，可以根据需要调整学习率


In [46]:
def train(model, iterator, optimizer, criterion):
    model.train()
    for batch in iterator:
        text, labels = batch
        optimizer.zero_grad()
        predictions = model(text)
        loss = criterion(predictions.squeeze(1), labels)
        loss.backward()
        optimizer.step()

def evaluate(model, iterator, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch
            predictions = model(text)
            loss = criterion(predictions.squeeze(1), labels)
            total_loss += loss.item()
    return total_loss / len(iterator)

# 训练模型
N_EPOCHS = 5  # 根据需要选择训练轮数
for epoch in range(N_EPOCHS):
    train(model, train_loader, optimizer, criterion)
    loss = evaluate(model, test_loader, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTest Loss: {loss:.3f}')


Epoch: 01
	Test Loss: 0.697
Epoch: 02
	Test Loss: 0.695
Epoch: 03
	Test Loss: 0.699
Epoch: 04
	Test Loss: 0.697
Epoch: 05
	Test Loss: 0.696


In [47]:
# 假设 model 是你的模型
torch.save(model.state_dict(), 'model_weights.pth')


In [48]:
# 加载已经训练好的模型权重
model.load_state_dict(torch.load('model_weights.pth'))

# 设置模型为评估模式
model.eval()

# 假设test_data是你的测试数据，是一个列表，每个元素都是数字化的句子


predictions = []  # 用于存储预测结果的列表

with torch.no_grad():
    for index, row in test_set.iterrows():
        text = torch.tensor(row['text'], dtype=torch.long).unsqueeze(0)  # 将数字列表转换为张量
        predictions_tensor = model(text)
        predicted_class = (predictions_tensor > 0.5).float()  # 这里假设阈值为0.5，可以根据需要调整
        predictions.append({"id": row['id'], "label": int(predicted_class.item())})


In [49]:
predictions[:10]

[{'id': 0, 'label': 0},
 {'id': 1, 'label': 0},
 {'id': 2, 'label': 1},
 {'id': 3, 'label': 0},
 {'id': 4, 'label': 0},
 {'id': 5, 'label': 1},
 {'id': 6, 'label': 0},
 {'id': 7, 'label': 0},
 {'id': 8, 'label': 1},
 {'id': 9, 'label': 0}]

In [50]:
output_csv_file = 'predictions.csv'

# 打开CSV文件并写入预测结果
with open(output_csv_file, 'w', newline='') as csvfile:
    fieldnames = ['id', 'class']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # 写入CSV文件的表头
    writer.writeheader()

    # 写入预测结果
    for prediction in predictions:
        writer.writerow({'id': prediction['id'], 'class': prediction['label']})

print(f'Predictions saved to {output_csv_file}')

Predictions saved to predictions.csv


In [51]:
result = pd.read_csv('predictions.csv')

In [52]:
result[result['class'] == 0]

,id,class
0,0,0
1,1,0
3,3,0
4,4,0
6,6,0
...,...,...
993,993,0
994,994,0
996,996,0
997,997,0
